In [4]:
# ! pip install imbalanced-learn
# ! pip install lime
# ! pip install textblob
# ! pip install contractions
# ! pip install spacy
# ! python -m spacy download en_core_web_sm
! pip install python-Levenshtein
! pip install gensim

     |████████████████████████████████| 50 kB 3.7 MB/s eta 0:00:011
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp36-cp36m-linux_x86_64.whl size=82218 sha256=424b5109ab6396490557b5818364f0d31cdd238544716353be3255a27722ba9d
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4a/a4/bf/d761b0899395c75fa76d003d607b3869ee47f5035b8afc30a2
Successfully built python-Levenshtein


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import pickle 
import time
from collections import Counter

# import lime
# import lime.lime_tabular
# from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.pipeline import Pipeline
# from imblearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, plot_confusion_matrix


from gensim.models import Word2Vec
import gensim

from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

## Load cleaned data
combine title and body of text 

In [2]:
aita_w2v = pd.read_csv('data/aita_w2v.csv')

# 1 - train test split

In [10]:
X = aita_w2v['combo_clean']
y = aita_w2v['is_asshole']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.01, random_state=21)
print(X_train.shape, X_test.shape)

(75267,) (761,)


In [4]:
Counter(y)

Counter({1: 20795, 0: 55233})

# 2 - word2vec

### 2.1 Create tokenized list, detect unigrams, bigrams, trigrams
### 2.2 Train word2vec model on list of list of unigrams, bigrams, trigrams

In [8]:
# %%time
# lst_corpus = [post.split() for post in X_train]
# bigrams_detector = gensim.models.Phrases(lst_corpus, min_count=5, threshold=10)
# trigrams_detector = gensim.models.Phrases(bigrams_detector[lst_corpus], min_count=5, threshold=10)
# lst_corpus = list(bigrams_detector[lst_corpus])
# lst_corpus = list(trigrams_detector[lst_corpus])
# nlp = gensim.models.word2vec.Word2Vec(lst_corpus, vector_size=200, window=4, min_count=3, sg=1)

CPU times: user 16min 26s, sys: 1.85 s, total: 16min 27s
Wall time: 7min 45s


In [11]:
# %%time
# ## tokenize text
# tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', oov_token="NaN", filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
# tokenizer.fit_on_texts(lst_corpus)

CPU times: user 12.6 s, sys: 7.98 ms, total: 12.7 s
Wall time: 12.7 s


Save and load models

In [12]:
# %%time
# nlp.save("models/nlp.model")
# bigrams_detector.save('models/bigrams_detector.pkl')
# trigrams_detector.save('models/trigrams_detector.pkl')
# with open('models/tokenizer.pkl', 'wb') as f:
#     pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)

CPU times: user 9.99 s, sys: 52 ms, total: 10 s
Wall time: 10 s


In [13]:
nlp = Word2Vec.load("models/nlp.model")
bigrams_detector = gensim.models.Phrases.load("models/bigrams_detector.pkl")
trigrams_detector = gensim.models.Phrases.load("models/trigrams_detector.pkl")
with open('models/tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [45]:
nlp_.wv.most_similar('angry')

[('upset', 0.8202987313270569),
 ('mad', 0.7894049286842346),
 ('pissed_off', 0.765598714351654),
 ('irate', 0.7474846243858337),
 ('frustrated', 0.741222083568573),
 ('annoyed', 0.7384644150733948),
 ('irritated', 0.7150590419769287),
 ('snippy', 0.7134074568748474),
 ('super_pissed_off', 0.7115201354026794),
 ('snappy', 0.7107669711112976)]

detect bigrams and trigrams from unigram lists

In [47]:
%%time
lst_corpus = [post.split() for post in X_train]
lst_corpus = list(bigrams_detector[lst_corpus])
lst_corpus = list(trigrams_detector[lst_corpus])

CPU times: user 1min 37s, sys: 536 ms, total: 1min 38s
Wall time: 1min 38s


In [48]:
## use fitted dectors on test set
lst_corpus_test = [post.split() for post in X_test]
lst_corpus_test = list(bigrams_detector[lst_corpus_test])
lst_corpus_test = list(trigrams_detector[lst_corpus_test])

### 2.3 tokenize text, create padded sequence

In [49]:
%%time
dic_vocabulary = tokenizer.word_index
## create sequence
lst_text2seq= tokenizer.texts_to_sequences(lst_corpus)
## padding sequence
X_train_p = kprocessing.sequence.pad_sequences(lst_text2seq, maxlen=400, padding="post", truncating="post")

CPU times: user 10.4 s, sys: 40 ms, total: 10.4 s
Wall time: 10.4 s


In [50]:
## text to sequence with the fitted tokenizer
lst_text2seq_test = tokenizer.texts_to_sequences(lst_corpus_test)
## padding sequence
X_test_p = kprocessing.sequence.pad_sequences(lst_text2seq_test, maxlen=400, padding="post", truncating="post")

CPU times: user 121 ms, sys: 0 ns, total: 121 ms
Wall time: 121 ms


In [51]:
print(len(lst_corpus),len(X_train_p))
print(len(lst_corpus_test),len(X_test_p))

75267 75267
761 761


In [52]:
%%time
## start the matrix (length of vocabulary x vector size) with all 0s
embeddings = np.zeros((len(dic_vocabulary)+1, 200))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp.wv[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

CPU times: user 261 ms, sys: 40 ms, total: 301 ms
Wall time: 300 ms


In [61]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

## code attention layer
def attention_layer(inputs, neurons):
    x = layers.Permute((2,1))(inputs)
    x = layers.Dense(neurons, activation="softmax")(x)
    x = layers.Permute((2,1), name="attention")(x)
    x = layers.multiply([inputs, x])
    return x

## input
x_in = layers.Input(shape=(400,))
## embedding
x = layers.Embedding(input_dim=embeddings.shape[0],  
                     output_dim=embeddings.shape[1], 
                     weights=[embeddings],
                     input_length=400, trainable=False)(x_in)
## apply attention
x = attention_layer(x, neurons=400)
## 2 layers of bidirectional lstm
x = layers.Bidirectional(layers.LSTM(units=50, dropout=0.2, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(units=50, dropout=0.2))(x)
## final dense layers
x = layers.Dense(64, activation='relu')(x)
y_out = layers.Dense(2, activation='softmax')(x)
## compile
model = models.Model(x_in, y_out)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy',precision_m,precision_m,recall_m])

model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 400)]        0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 400, 200)     29763600    input_5[0][0]                    
__________________________________________________________________________________________________
permute_4 (Permute)             (None, 200, 400)     0           embedding_4[0][0]                
__________________________________________________________________________________________________
dense_10 (Dense)                (None, 200, 400)     160400      permute_4[0][0]                  
____________________________________________________________________________________________

In [58]:
X_train_p.shape,y_train.size

((75267, 400), 75267)

In [ ]:
%%time
## encode y
dic_y_mapping = {n:label for n,label in enumerate(np.unique(y_train))}
inverse_dic = {v:k for k,v in dic_y_mapping.items()}
y_train_i = np.array([inverse_dic[y] for y in y_train])

## train
training = model.fit(x=X_train_p, y=y_train_i, batch_size=200, 
                     epochs=10, shuffle=True, verbose=0, 
                     validation_split=0.3)

# save model and architecture to single file
training.save("models/keras_trained.h5")
print("Saved model to disk")

In [ ]:
## plot loss and accuracy
metrics = [k for k in tr
           aining.history.keys() if ("loss" not in k) and ("val" not in k)]
fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True)
ax[0].set(title="Training")
ax11 = ax[0].twinx()
ax[0].plot(training.history['loss'], color='black')
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Loss', color='black')
for metric in metrics:
    ax11.plot(training.history[metric], label=metric)
ax11.set_ylabel("Score", color='steelblue')
ax11.legend()
ax[1].set(title="Validation")
ax22 = ax[1].twinx()
ax[1].plot(training.history['val_loss'], color='black')
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss', color='black')
for metric in metrics:
     ax22.plot(training.history['val_'+metric], label=metric)
ax22.set_ylabel("Score", color="steelblue")
plt.show()

In [ ]:
## test
predicted_prob = model.predict(X_test_p)
predicted = [dic_y_mapping[np.argmax(pred)] for pred in predicted_prob]

In [6]:
def remove_punc_stop(s):
    s = re.sub('[%s]' % re.escape(string.punctuation), '', s)
    s = re.sub('[‘’“”…]', '', s)
    s = re.sub('\w*\d\w*', '', s)
    return s

In [7]:
def make_feature_vec(words, model, num_features):
    """
    Average the word vectors for a set of words
    """
    feature_vec = np.zeros((num_features,),dtype="float32")  # pre-initialize (for speed)
    nwords = 0.
    index2word_set = set(model.wv.index_to_key)  # words known to the model
    stop = set(stopwords.words('english')).union(['aita','wibta'])
    for word in words:
        if word in index2word_set and word not in stop: 
            nwords = nwords + 1.
            feature_vec = np.add(feature_vec,model.wv[word])
    feature_vec = np.divide(feature_vec, nwords)
    return feature_vec
def get_avg_feature_vecs(texts, model, num_features):
    """
    Calculate average feature vectors for all reviews
    """
    feature_vecs = np.zeros((len(texts),num_features), dtype='float32')  # pre-initialize (for speed)
    for ix, text in enumerate(texts):
        feature_vecs[ix] = make_feature_vec(text, model, num_features)
    return feature_vecs

In [135]:
# feature_vecs = get_avg_feature_vecs(aita_w2v['tokens'], model, 100)
# feature_vecs_df = pd.DataFrame(feature_vecs, columns=[f'w2v_{i}' for i in range(1, 101)])
# aita_w2v_1 = pd.concat([aita_w2v.reset_index(drop=True),feature_vecs_df], axis=1)

In [8]:
# aita_w2v_1.to_csv('data/aita_w2v_1.csv', index=False)
# aita_w2v_1 = pd.read_csv('data/aita_w2v_1.csv')

In [12]:
nlp.wv.most_similar(positive=['york'])

[('zealand', 0.9204254150390625),
 ('orleans', 0.8691218495368958),
 ('jersey', 0.744202733039856),
 ('mexico', 0.7416518330574036),
 ('england', 0.7132971286773682),
 ('hires', 0.7053131461143494),
 ('hampshire', 0.6276334524154663),
 ('boston', 0.6179364323616028),
 ('wales', 0.6043822765350342),
 ('city', 0.5941430926322937)]

## 4 Oversample minority class with SMOTE

In [162]:
# sentiment analysis + vectors
X_train_smote, y_train_smote = SMOTE().fit_resample(X_train, y_train)
X_test_smote, y_test_smote = SMOTE().fit_resample(X_test, y_test)
Counter(y_train_smote), Counter(y_test_smote)

(Counter({0: 54680, 1: 54680}), Counter({1: 553, 0: 553}))